# VacationPy
---

## Starter Code to Import Libraries and Load the Weather and Coordinates Data

In [1]:
# Dependencies and Setup
import hvplot.pandas
import pandas as pd
import requests

# Import API key
from api_keys import geoapify_key

In [2]:
# Load the CSV file created in Part 1 into a Pandas DataFrame
city_data_df = pd.read_csv("output_data/cities.csv")

# Display sample data
city_data_df.head()

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,0,adamstown,-25.0660,-130.1015,21.00,74,29,7.49,PN,1722723842
1,1,port-aux-francais,-49.3500,70.2167,1.90,90,23,11.83,TF,1722723843
2,2,punta arenas,-53.1500,-70.9167,0.62,80,100,4.19,CL,1722723844
3,3,iqaluit,63.7506,-68.5145,4.85,80,20,7.72,CA,1722723789
4,4,ilulissat,69.2167,-51.1000,11.01,54,4,0.00,GL,1722723846


---

### Step 1: Create a map that displays a point for every city in the `city_data_df` DataFrame. The size of the point should be the humidity in each city.

In [16]:
%%capture --no-display

# Configure the map plot
plot_1 = city_data_df.hvplot.points(
    'Lng',
    'Lat',
    geo=True, 
    tiles = 'OSM', 
    frame_width = 700, 
    frame_height = 500, 
    color='City',
    size = "Humidity",
    scale= 0.8)


# Display the map
plot_1

:Overlay
   .WMTS.I   :WMTS   [Longitude,Latitude]
   .Points.I :Points   [Lng,Lat]   (City,Humidity)

### Step 2: Narrow down the `city_data_df` DataFrame to find your ideal weather condition

In [5]:
city_data_df

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,0,adamstown,-25.0660,-130.1015,21.00,74,29,7.49,PN,1722723842
1,1,port-aux-francais,-49.3500,70.2167,1.90,90,23,11.83,TF,1722723843
2,2,punta arenas,-53.1500,-70.9167,0.62,80,100,4.19,CL,1722723844
3,3,iqaluit,63.7506,-68.5145,4.85,80,20,7.72,CA,1722723789
4,4,ilulissat,69.2167,-51.1000,11.01,54,4,0.00,GL,1722723846
...,...,...,...,...,...,...,...,...,...,...
549,549,shalkar,47.8288,59.6258,21.51,56,30,2.29,KZ,1722724485
550,550,murray bridge,-35.1167,139.2667,9.08,72,100,1.21,AU,1722724487
551,551,al burayqah,30.4062,19.5739,27.08,66,0,3.63,LY,1722724488
552,552,diok'nisi,41.6293,42.3917,16.59,95,19,2.03,GE,1722724489


In [10]:
# Narrow down cities that fit criteria and drop any results with null values
ideal_df = city_data_df.loc[(city_data_df['Max Temp']< 27)\
                            &(city_data_df['Max Temp']> 21)\
                            &(city_data_df['Cloudiness']== 0)\
                            &(city_data_df['Wind Speed'] < 4.5),:]
print(len(ideal_df))

# Drop any rows with null values
ideal_df.dropna()

# Display sample data
ideal_df.head()

15


,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
26,26,ribeira grande,38.5167,-28.7000,23.21,87,0,1.34,PT,1722723871
77,77,pamandzi,-12.7967,45.2794,21.88,88,0,2.06,YT,1722723931
89,89,the pas,53.8251,-101.2541,22.01,40,0,2.57,CA,1722723945
140,140,marivan,35.5219,46.1760,23.99,30,0,1.40,IR,1722724007
175,175,targuist,34.9377,-4.3186,22.54,30,0,1.48,MA,1722724048


### Step 3: Create a new DataFrame called `hotel_df`.

In [13]:
# Use the Pandas copy function to create DataFrame called hotel_df to store the city, country, coordinates, and humidity
hotel_df = ideal_df.iloc[:,[1,8,2,3,5]].copy()

# Add an empty column, "Hotel Name," to the DataFrame so you can store the hotel found using the Geoapify API
hotel_df['Hotel Name']=''

# Display sample data
hotel_df.head()

,City,Country,Lat,Lng,Humidity,Hotel Name
26,ribeira grande,PT,38.5167,-28.7000,87,
77,pamandzi,YT,-12.7967,45.2794,88,
89,the pas,CA,53.8251,-101.2541,40,
140,marivan,IR,35.5219,46.1760,30,
175,targuist,MA,34.9377,-4.3186,30,


### Step 4: For each city, use the Geoapify API to find the first hotel located within 10,000 metres of your coordinates.

In [26]:
# Set parameters to search for a hotel
radius = 10000
# using places api
params = {
    'categories': 'accommodation.hotel',
    'filter': f'circle:{longitude},{latitude},{radius}',
    'apiKey': api_key,
    'limit' : 1,
    'bias' : ''
}
# Print a message to follow up the hotel search
print("Starting hotel search")

# Iterate through the hotel_df DataFrame
for index, row in hotel_df.iterrows():
    # get latitude, longitude from the DataFrame
    lon = row['Lng']
    lat = row['Lat']

    # Add the current city's latitude and longitude to the params dictionary
    params["filter"] = f'circle:{lon},{lat},{radius}'
    params["bias"] = f'proximity:{lon},{lat}'

    # Set base URL
    base_url = "https://api.geoapify.com/v2/places"
    
    # Make and API request using the params dictionary
    name_address = requests.get(base_url, params=params)
   
    # Convert the API response to JSON format
    name_address = name_address.json()

    # Grab the first hotel from the results and store the name in the hotel_df DataFrame
    try:
        hotel_df.loc[index, "Hotel Name"] = name_address["features"][0]["properties"]["name"]
    except (KeyError, IndexError):
        # If no hotel is found, set the hotel name as "No hotel found".
        hotel_df.loc[index, "Hotel Name"] = "No hotel found"

    # Log the search results
    print(f"{hotel_df.loc[index, 'City']} - nearest hotel: {hotel_df.loc[index, 'Hotel Name']}")

# Display sample data
hotel_df

Starting hotel search
ribeira grande - nearest hotel: Hospedaria JSF
pamandzi - nearest hotel: Gite Le Madina
the pas - nearest hotel: Wescana Inn
marivan - nearest hotel: هتل میران
targuist - nearest hotel: No hotel found
lagoa - nearest hotel: Casa da Madrinha
annaba - nearest hotel: فندق الهنا
gurupi - nearest hotel: Via Norte
moires - nearest hotel: Hotel Olympic
edeia - nearest hotel: Hotel Roma
guri-si - nearest hotel: 첼로호텔
poxoreo - nearest hotel: No hotel found
rosetta - nearest hotel: No hotel found
kinkala - nearest hotel: hotel
sarykemer - nearest hotel: No hotel found


,City,Country,Lat,Lng,Humidity,Hotel Name
26,ribeira grande,PT,38.5167,-28.7000,87,Hospedaria JSF
77,pamandzi,YT,-12.7967,45.2794,88,Gite Le Madina
89,the pas,CA,53.8251,-101.2541,40,Wescana Inn
140,marivan,IR,35.5219,46.1760,30,هتل میران
175,targuist,MA,34.9377,-4.3186,30,No hotel found
197,lagoa,PT,39.0500,-27.9833,80,Casa da Madrinha
320,annaba,DZ,36.9000,7.7667,73,فندق الهنا
359,gurupi,BR,-11.7292,-49.0686,33,Via Norte
365,moires,GR,35.0532,24.8744,61,Hotel Olympic
379,edeia,BR,-17.3383,-49.9314,29,Hotel Roma


### Step 5: Add the hotel name and the country as additional information in the hover message for each city in the map.

In [28]:
%%capture --no-display

# Configure the map plot
plot_2 = hotel_df.hvplot.points(
    'Lng',
    'Lat',
    geo=True, 
    tiles = 'OSM', 
    frame_width = 700, 
    frame_height = 500, 
    color='City',
    size = "Humidity",
    scale= 0.8,
    hover=True,
    hover_cols=['Hotel Name', 'City', 'Country', 'Humidity'])


# Display the map
plot_2


:Overlay
   .WMTS.I   :WMTS   [Longitude,Latitude]
   .Points.I :Points   [Lng,Lat]   (City,Humidity,Hotel Name,Country)